# Ridecheck Data Manipulator
#### This jupyter notebook generates ridership numbers to join in the bus stop inventory.  It filters out 911, 912, 913 and 914, and then pivots and sums the boarding and alighting numbers and adds 0's to the empty spaces

In [175]:
import pandas as pd
import glob
from amigocloud import AmigoCloud

In [176]:
# allFiles = glob.glob('.' + "/gps_*.csv")
frame = pd.DataFrame()
frames = []
for file_ in ['Ridership/WEEKDAY.XLSX','Ridership/LRTWEEKDAY.XLSX']:
    df = pd.read_excel(file_, header=0)
    df['DAY']='RS_WKDY'
    #     df['DAY']='Weekday'
    frames.append(df)
for file_ in ['Ridership/SATURDAY.XLSX','Ridership/LRTSATURDAY.XLSX']:
    df = pd.read_excel(file_, header=0)
    df['DAY']='RS_SAT'
    frames.append(df)
for file_ in ['Ridership/SUNDAY.XLSX','Ridership/LRTSUNDAY.XLSX']:
    df = pd.read_excel(file_, header=0)
    df['DAY']='RS_SUN'
    frames.append(df)

df = pd.concat(frames)
df = df[~df['ROUTE_NUMBER'].isin([911,912,913,914])]

In [183]:
# wb_lrt
ridership_by_stop = df.groupby(['STOP_ID','DAY'])['BOARD_ALL', 'ALIGHT_ALL'].sum().round()

In [185]:
ridership_by_stop.columns

Index(['BOARD_ALL', 'ALIGHT_ALL'], dtype='object')

In [186]:
pd.pivot_table(ridership_by_stop.reset_index(), index=["STOP_ID"],values=["BOARD_ALL",'ALIGHT_ALL'],
               columns="DAY").columns

MultiIndex(levels=[['BOARD_ALL', 'ALIGHT_ALL'], ['RS_SAT', 'RS_SUN', 'RS_WKDY']],
           labels=[[0, 0, 0, 1, 1, 1], [0, 1, 2, 0, 1, 2]],
           names=[None, 'DAY'])

In [187]:
pd.pivot_table(ridership_by_stop.reset_index(), index=["STOP_ID"],values=["BOARD_ALL",'ALIGHT_ALL'],
               columns="DAY").reset_index().columns

MultiIndex(levels=[['BOARD_ALL', 'ALIGHT_ALL', 'STOP_ID'], ['RS_SAT', 'RS_SUN', 'RS_WKDY', '']],
           labels=[[2, 0, 0, 0, 1, 1, 1], [3, 0, 1, 2, 0, 1, 2]],
           names=[None, 'DAY'])

In [188]:
unstack = pd.pivot_table(ridership_by_stop.reset_index(), index=["STOP_ID"],values=["BOARD_ALL",'ALIGHT_ALL'],
               columns="DAY").unstack().reset_index()

In [189]:
unstack['desc'] = unstack['DAY'] + '_' + unstack['level_0'].str.slice(0,1)

In [192]:
#This produces the usable ridership numbers

pd.pivot_table(unstack, index=["STOP_ID"],values=unstack[0],
               columns="desc").reset_index(col_level=1).to_csv("OCT_board_alight_By_Day_By_Stop_delete_header.csv", index=False, na_rep='0')

# The rest of this is about multiindex exploration

In [166]:
labeled.reset_index().columns

MultiIndex(levels=[[0, 'STOP_ID'], ['RS_SAT_A', 'RS_SAT_B', 'RS_SUN_A', 'RS_SUN_B', 'RS_WKDY_A', 'RS_WKDY_B', '']],
           labels=[[1, 0, 0, 0, 0, 0, 0], [6, 0, 1, 2, 3, 4, 5]],
           names=[None, 'desc'])

In [149]:
labeled.columns

MultiIndex(levels=[[0], ['RS_SAT_A', 'RS_SAT_B', 'RS_SUN_A', 'RS_SUN_B', 'RS_WKDY_A', 'RS_WKDY_B']],
           labels=[[0, 0, 0, 0, 0, 0], [0, 1, 2, 3, 4, 5]],
           names=[None, 'desc'])

In [123]:
list(labeled)

[('', 'STOP_ID'),
 (0, 'RS_SAT_A'),
 (0, 'RS_SAT_B'),
 (0, 'RS_SUN_A'),
 (0, 'RS_SUN_B'),
 (0, 'RS_WKDY_A'),
 (0, 'RS_WKDY_B')]

In [76]:
pd.pivot_table(unstack, index=["STOP_ID"],values=unstack[0],
               columns="desc").reset_index().to_csv("OCT_board_alight_By_Day_By_Stop.csv", index=False)

In [36]:
ridership_by_stop

STOP_ID BOARD_ALL                ALIGHT_ALL                
DAY             RS_SAT RS_SUN RS_WKDY     RS_SAT  RS_SUN RS_WKDY
0          1      84.0   87.0   126.0       98.0    96.0   111.0
1          8     101.0  120.0   137.0       79.0    94.0   128.0
2         11     151.0  232.0   220.0      163.0   147.0   198.0
3         12       1.0    2.0     6.0       10.0     9.0    22.0
4         13       1.0    0.0     2.0        4.0     6.0     7.0
5         14     189.0  214.0   306.0      169.0   234.0   273.0
6         15       8.0   11.0    34.0        7.0    14.0    26.0
7         16       3.0    6.0    19.0        0.0     1.0    10.0
8         19       NaN    NaN     4.0        NaN     NaN     0.0
9         20      29.0   27.0    43.0       31.0    28.0    48.0
10        21     166.0  156.0   269.0      206.0   192.0   444.0
11        22     100.0   97.0   252.0       29.0    32.0    72.0
12        23       2.0    1.0     4.0        4.0     2.0    10.0
13        24       3.0    1.0     5.0        9.0     7.0    16.0
14        25       1.0    2.0     3.0        4.0     3.0     6.0
15        26       2.0    4.0    10.0       13.0     8.0    60.0
16        27       4.0    4.0     4.0        5.0     5.0    13.0
17        28      27.0   26.0    23.0       34.0    22.0    47.0
18        29       7.0    8.0     5.0        6.0     5.0    10.0
19        30     105.0  115.0     6.0       59.0    65.0     7.0
20        31      47.0   34.0    54.0       11.0    13.0    10.0
21        32      30.0   30.0    57.0       13.0    12.0    18.0
22        33      18.0    8.0    93.0        7.0     4.0    26.0
23        34      42.0   28.0    71.0       34.0    29.0    54.0
24        35     156.0  136.0   209.0      153.0   148.0   199.0
25        36      35.0   41.0    51.0       33.0    34.0    50.0
26        37      37.0   32.0    65.0       31.0    37.0    82.0
27        38      28.0   23.0    43.0       44.0    38.0    58.0
28        39       4.0    2.0     6.0       46.0    35.0    75.0
29        40     229.0  191.0   293.0      367.0   321.0   432.0
...      ...       ...    ...     ...        ...     ...     ...
3751    5804       NaN    NaN     5.0        NaN     NaN     7.0
3752    5805       NaN    NaN    58.0        NaN     NaN    95.0
3753    5806      75.0   74.0   139.0       69.0    68.0   135.0
3754    5807     144.0  116.0   175.0      134.0   107.0   169.0
3755    5808     129.0  111.0   150.0        2.0     1.0     1.0
3756    5809     169.0  130.0   319.0        0.0     1.0     1.0
3757    5810     872.0  730.0  1095.0        5.0     9.0     9.0
3758    5812      12.0   14.0    17.0     1249.0  1059.0  1618.0
3759    5814       NaN    NaN    10.0        NaN     NaN     1.0
3760    5815       4.0    3.0     6.0        8.0     4.0    14.0
3761    5822      22.0   14.0    29.0       37.0    28.0    56.0
3762    5823       0.0    0.0     0.0        0.0     0.0     0.0
3763    5824       0.0    1.0     1.0        0.0     1.0     1.0
3764    5829      14.0   12.0    31.0       55.0    44.0    70.0
3765    5830       0.0    NaN     2.0        0.0     NaN     7.0
3766    5831       0.0    NaN     7.0        1.0     NaN     3.0
3767    5837       0.0    NaN     3.0        2.0     NaN     4.0
3768    5843       6.0    NaN    17.0       13.0     NaN    60.0
3769    5846      75.0   53.0   134.0      103.0    58.0   195.0
3770    5847      81.0   53.0   162.0       65.0    41.0   105.0
3771    5849     407.0  387.0   430.0      367.0   321.0   410.0
3772    5850     171.0  103.0   315.0      278.0   158.0   469.0
3773    5856      25.0   19.0    45.0       67.0    48.0   129.0
3774    5857      56.0   40.0   112.0       18.0    13.0    36.0
3775    5858      27.0   19.0    44.0       81.0    55.0   158.0
3776    5859      77.0   58.0   175.0       26.0    20.0    43.0
3777    5860       NaN    NaN    11.0        NaN     NaN    16.0
3778    5861       NaN    NaN     5.0        NaN     NaN     4.0
3779    5864       NaN    NaN     0.0   

In [37]:
ridership_by_stop['BOARD_ALL'].head()

DAY,RS_SAT,RS_SUN,RS_WKDY
0,84.0,87.0,126.0
1,101.0,120.0,137.0
2,151.0,232.0,220.0
3,1.0,2.0,6.0
4,1.0,0.0,2.0


In [38]:
ridership_by_stop['ALIGHT_ALL'].head()

DAY,RS_SAT,RS_SUN,RS_WKDY
0,98.0,96.0,111.0
1,79.0,94.0,128.0
2,163.0,147.0,198.0
3,10.0,9.0,22.0
4,4.0,6.0,7.0


In [39]:
list(ridership_by_stop)

[('STOP_ID', ''),
 ('BOARD_ALL', 'RS_SAT'),
 ('BOARD_ALL', 'RS_SUN'),
 ('BOARD_ALL', 'RS_WKDY'),
 ('ALIGHT_ALL', 'RS_SAT'),
 ('ALIGHT_ALL', 'RS_SUN'),
 ('ALIGHT_ALL', 'RS_WKDY')]